In [ ]:
import tensorflow as tf
class LateSpatialDropout1D(tf.keras.layers.Layer):
    def __init__(self, rate, start_step=0, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.rate = rate
        self.start_step = start_step
        self.spatial_dropout = tf.keras.layers.SpatialDropout1D(rate)
      
    def build(self, input_shape):
        super().build(input_shape)
        agg = tf.VariableAggregation.ONLY_FIRST_REPLICA
        self._train_counter = tf.Variable(0, dtype="int64", aggregation=agg, trainable=False)

    def call(self, inputs, training=False):
        x = tf.cond(self._train_counter < self.start_step, 
                    lambda: inputs, 
                    lambda: self.spatial_dropout(inputs, training=training))
        
        if training:
            self._train_counter.assign_add(1)
        
        return x
